# Structuur: JSON-schema

* MongoDB/document-database: elk document (in een collection) bepaalt zijn eigen structuur
* voor zoeken in een collection: (deels) gemeenschappelijke structuur van documenten nodig
* met JSON-schema kun je de *structuur van een document beschrijven*; je kunt daarmee controleren of een bepaald document aan dit schema voldoet; je documenteert daarmee ook de collection
* we noemen een document dat aan een JSON-schema voldoet ook wel een *instantie* (instance) van dat schema
* je kunt een JSON-schema ook zien als meta-data: het is data die de structuur van andere data (documenten) beschrijft.
* in MongoDB kun je een JSON-schema koppelen aan een collection, om af te dwingen dat alle documenten in die collection aan dat schema voldoen.
* JSON-schema dit is niet alleen zinvol voor (document) databases, maar bijvoorbeeld ook voor:
    * API's die JSON-documenten als invoer verwachten, of als resultaat opleveren;
    * UI voor formulierinvoer van documenten 
    * (andere contexten waar JSON documenten gebruikt worden?)

## Inleiding

Zoals je eerder gezien hebt, bepaalt elk (JSON-)document in een MongoDB collection zijn eigen structuur: de namen en types van de velden.
Voor een zoekopdracht in een collection, bijvoorbeeld naar documenten met `plaats = "Amsterdam"`, is het wel zo handig als alle documenten een veld `plaats` hebben, met een string als waarde. Een document met `{..."city": "Amsterdam"...}` komt immers niet in het zoekresultaat voor.

Met een *JSON-schema* (https://json-schema.org) kun je controleren (valideren) of een JSON-document (ten minste) bepaalde velden heeft, met de juiste namen en types.
Door een JSON-schema te koppelen aan een MongoDB *collection* kun je afdwingen dat alleen documenten die aan het schema voldoen aan die collection toegevoegd worden.

```{admonition} Opmerking
Je kunt een JSON-schema vergelijken met een *type* of een *class* voor JSON-documenten.
Een JSON-document (instantie) is dan een waarde van dat type, of object van die class.
```

We geven eerst een elementair JSON-schema voorbeeld. Voor veel programmeertalen zijn er libraries om te werken met JSON-schema. We gebruiken hier de Python-library `jsonschema`, zie https://python-jsonschema.readthedocs.io.

In [1]:
from jsonschema import validate

Dit eerste voorbeeld geeft een schema met twee velden, `naam` en `leeftijd`. Voor elk veld kun je het type aangeven. Met behulp van `required` geef je aan welke velden *verplicht* zijn.

In [2]:
schema = {
    "type": "object",
    "properties": {
        "naam": {"type": "string"},
        "leeftijd": {"type": "integer"}
    },
    "required": ["naam"]
}

Met `validate` kun je controleren of een JSON-object voldoet aan dit schema.
(Alleen als dat niet het geval is krijg je een foutmelding (exception); anders niets.)

In [3]:
object1 = {"naam": "Hans", "leeftijd": 33}
validate(instance=object1, schema=schema)

In het volgende voorbeeld klopt de waarde van het veld `leeftijd` niet met het type zoals gegeven in het schema. Dit resulteert in een foutmelding.

In [4]:
object2 = {"naam": "Kees", "leeftijd": "dertig"}
validate(instance=object2, schema=schema)

ValidationError: 'dertig' is not of type 'integer'

Failed validating 'type' in schema['properties']['leeftijd']:
    {'type': 'integer'}

On instance['leeftijd']:
    'dertig'

Bij de volgende instantie ontbreekt het verplichte veld "naam", waarschijnlijk door de verwarring met "voornaam":

In [5]:
object3 = {"voornaam": "Marie", "leeftijd": 21}
validate(instance=object3, schema=schema)

ValidationError: 'naam' is a required property

Failed validating 'required' in schema:
    {'properties': {'leeftijd': {'type': 'integer'},
                    'naam': {'type': 'string'}},
     'required': ['naam'],
     'type': 'object'}

On instance:
    {'leeftijd': 21, 'voornaam': 'Marie'}

Voorbeelden:

* beschrijving van de structuur van een Person (contact)
* beschrijving van de structuur van een Event
* beschrijving van de structuur van een Product (generalisatie/specialisatie)

Het voorbeeld hierboven is een eenvoudig begin. In JSON-schema kun je aangeven:

* wat de structuur is: `object`, `array`, of een enkelvoudig type;
* welke velden een object kan hebben; of moet hebben (`required`);
* welk type elk veld moet hebben; of elk item van een `array`;
* welke alternatieven er zijn (`oneOf`, `anyOf`);
* aan welke eisen de waarden moeten voldoen, bijvoorbeeld de minimum- en maximumwaarden;
* ...

In het volgende hoofdstuk geven we een aantal voorbeeld waarin meer van de mogelijkheden gebruikt worden.

## JSON-schema en MongoDB

Zoals gezegd moet je ervoor kunnen zorgen dat de documenten in een MongoDB collection eenzelfde gemeenschappelijke (deel)structuur hebben, om zoekopdrachten op een zinvolle manier te kunnen gebruiken.
In principe kun je met `jsonchema.validate` er al voor zorgen dat een document dat je toevoegt aan het gemeenschappelijke schema voldoet. Maar MongoDB maakt het nog wat gemakkelijker: je kunt een JSON-schema koppelen aan een collection. MongoDB zorgt er dan voor dat er alleen documenten toegevoegd kunnen worden die aan dat schema voldoen.

Voorbeelden:

* zoeken in documenten waarbij sommige documenten een andere structuur hebben, bijvoorbeeld gebruik van andere namen (tikfout en/of andere taal)
    * (dit moet in de voorbeeld-invoer voorbereid worden)
    * bij voorkeur in het vorige hoofdstuk
* beschrijven van een JSON-documentstructuur, en het controleren van enkele voorbeelden (bijv. de personen uit het database-voorbeeld?
* MongoDB: koppelen van JSON-schema aan een collection;
    * toevoegen van een document dat aan het schema voldoet
    * toevoegen van een document dat niet aan het schema voldoet


Volgende stap: gebruik van standaard-namen voor entiteiten/documenten, attributen/properties. Bijvoorbeeld: via schema.org

Dit is ook een opstap naar Linked Data. Schema.org is gebaseerd op de Linked Data principes.

## Voorbeeld: product met specialisaties

In één van de vorige hoofdstukken heb je gezien dat we verschillende varianten (specialisaties) van product-documenten in eenzelfde collection onderbrachten.
Je kunt in die collection dan op verschillende manieren zoeken: (i) op de algemene eigenschappen van een product; (ii) op de eigenschappen van een bepaalde specialisatie.

> De structuur van Product met specialisaties heet wel generalisatie-specialisatie: "Product" is de generalisatie, "Boek" en "Cd" zijn voorbeelden van specialisaties. In object-georiënteerde programmeertalen heet dit ook wel "overerving" of "inheritance": de specialisaties Boek en Cd erven de algemene eigenschappen (velden) van de generalisatie Product.

In JSON-schema kun je deze structuur als volgt weergeven:

```{margin} Gen/Spec in SQL
In SQL tabellen kun je de generalisatie/specialisatie-structuur weergeven door een verwijzing van het specialisatie-deel naar het generalisatie-deel. Voor de generalisatie en voor de verschillende specialisaties gebruik je aparte tabellen.
```

**Vraag**: het is duidelijk wat met `required` bedoeld wordt. De niet-required velden zijn optioneel. Zijn er buiten die velden nog andere velden mogelijk? Hoe geef je dat aan?

## Wat kun je nog meer met JSON-schema?

### Genereren van web-formulieren

Een formulier is een voorbeeld van gestructureerde data.
Je kunt de structuur van een formulier beschrijven met JSON-schema.
Met behulp van een generator-programma kun je dan de code genereren voor het bijbehorende web-formulier.
Er zijn generatoren voor allerlei web-frameworks, zoals React.

Zie bijvoorbeeld: https://rjsf-team.github.io/react-jsonschema-form/. Die website bevat ook allerlei voorbeelden van schema's en de bijbehorende formulieren.

### Documenteren en gebruiken van API's

JSON wordt vaak gebruikt in web-APIs, zowel voor de invoer-data van een web-request als voor de resulterende uitvoer.
Je kunt het formaat van de invoer- en uitvoer-documenten beschrijven met JSON-schema.
Dit is niet alleen zinvol als documentatie, maar je kunt ook de code voor het testen en verwerken uit de JSON laten genereren.

Voor een volledige beschrijving van een web-API moet je ook de paden (bijvoorbeeld "/products") en HTTP methods ("GET", "PUT", enz.) beschrijven. Hiervoor kun je OpenAPI gebruiken (https://swagger.io/specification/). Het deel van de OpenAPO specificatie dat betrekking heeft op de data die uitgewisseld wordt, is gebaseerd op JSON-schema. Met de Swagger hulpmiddelen kun je OpenAPI interfaces documenteren en testen.

Zie bijvoorbeeld:

* 
* https://json.schemastore.org/openweather.current.json (JSON-schema voor openweather API)
** https://openweathermap.org/current#current_JSON
* https://swagger.io
** https://swagger.io/tools/swagger-editor/

Voor API's is er de OpenAPI notatie, die bijvoorbeeld gebruikt wordt door Swagger (swagger.io). Daarmee kun je API's documenteren en controleren. Er zijn aparte programma's om JSON-schema om te zetten naar OpenAPI notatie.


## Opdrachten

* gegeven een JSON-Schema, breid dit uit met een veld; en maak twee voorbeelden, één van een document dat hieraan voldoet en één van een document dat (net niet) voldoet - bijvoorbeeld doordat de waarde niet van het juiste type is.
* gegeven een JSON-Schema, pas de documenten aan zodat deze aan het schema voldoen.
* MongoDB: koppel een JSON-Schema aan een collection, en controleer of de documenten aan dat schema voldoen. Zorg ervoor dat de documenten aan het schema voldoen.
* genereren van een formulier (HTML? Angular etc.?) vanuit JSON-Schema
* maak een formulier (met het React-tool) voor het onderstaande JSON-Schema
    * uitproberen van verschillende mogelijkheden
    * ook voor valideren van input!

### Toetsvragen

Met JSON schema

* kun je de structuur van een verzameling JSON-documenten (instanties) beschrijven
* kun je de structuur van een JSON-document valideren
* kun je de structuur van een JSON-document veranderen
* kun je de waarden in een JSON-document veranderen

Door een JSON schema aan een MongoDB collection te koppelen

* dwing je af dat de documenten in die collection precies dezelfde structuur hebben
* dwing je af dat de documenten in die collection een gemeenschappelijke deel-structuur hebben
* zorg je ervoor dat een aantal queries over die collection goed gedefinieerd zijn
* documenteer je de collection - door aan te geven welke documenten deze kan bevatten

## Gevorderde onderwerpen

Deze onderwerpen behandelen we in de keuzemodule.

* nesting van schema's
* gebruik van arrays, lijsten
* gebruik van alternatieven (-> support voor Product, Gen/Spec?)
* gebruik van JSON-schema voor API'a
* gebruik van JSON-schema voor web-formulieren

Verder lezen:
    
* https://json-schema.org/understanding-json-schema/